# Using SyGNet

This notebook demonstrates the basic functionality of the **sygnet** package in Python.

To download the package, simply run `pip install sygnet` at the command line.

## Prerequisites

To start, we load the required packages and define a data generating process (DGP).

In [1]:
%%capture
%cd ..

import pandas as pd
import numpy as np
from numpy.random import default_rng

# NB: once installed via pip, can run `from sygnet import SygnetModel`
from src.sygnet.sygnet_interface import SygnetModel


Next, we will focus on a very simple case of learning a parametric relationship between numeric variables.The DGP comprises two random uniform variables (`x1` and `x2`), which in turn impact the value of two other variables (`x3` and `y`). Once defined, we take 100,000 draws from this DGP to use as our training data:

In [2]:
rng = default_rng()

def gen_sim_data(rng, n=100000):
    
    x1 = rng.uniform(low = 0, high = 1, size = n)
    x2 = rng.uniform(low = 0, high = 1, size = n)
    x3 = rng.normal(loc = x1 + x2, scale = 0.1)
    y = rng.normal(loc=3*x1 + 2*x2 + 1, scale = 1)

    sim_data = pd.DataFrame({
        'x1' : x1,
        'x2' : x2,
        'x3' : x3,
        'y' : y
    })   

    return sim_data

train_data = gen_sim_data(rng)
train_data.head()

,x1,x2,x3,y
0,0.955283,0.750947,1.577567,6.774734
1,0.959211,0.375207,1.438572,4.508613
2,0.478289,0.827375,1.194855,2.594680
3,0.332720,0.127708,0.558522,0.464812
4,0.122932,0.066068,0.077215,2.008986


## **sygnet** pipeline

We follow a very similar pipeline to sci-kit learn: 

1. Instantiate a model:
   * Users must specify what type of GAN architecture to use: we recommend "wgan" for non-conditional synthetic data, and "cgan" when conditional labels will be supplied 
   * Optional arguments allow the user to customise the hidden layer structure, dropout proportions, layer norming, ReLU leakage, and whether to range match the final output
2. Fit the model to the training data
   * Users must supply the training data
   * Optional arguments allow the user to alter the default hyperparameters (epochs, learning rate, batch size etc.)
3. Sample from the trained model
   * Users must specify the number of synthetic observations to draw from the model
   * Optional arguments allow the user to control the format of the returned results, as well as to save the synthetic data to disk



## Basic example

In this first example, we set `mode = "wgan"` to use the Wassterstein GAN architecture. We fit the model to our simulated data, for a single epoch (real uses will require more epochs), and then generate 100 synthetic observations:

In [3]:
model = SygnetModel(mode = "wgan")
model.fit(data = train_data, epochs = 1)
synth_data = model.sample(nobs = 1000)

synth_data.head()

Epoch: 100%|██████████| 1/1 [00:07<00:00,  7.59s/it]


,x1,x2,x3,y
0,-0.261301,-1.672521,0.000000,0.986449
1,0.621772,-2.204374,0.034815,1.836472
2,-0.281299,-1.651041,0.034763,1.578335
3,-0.038010,-1.524639,0.000000,0.873217
4,-0.058963,-1.948267,0.759016,0.864480


### GPU support

**sygnet** allows users to train the model using GPU computation, which should improve training times considerably. To run the synthetic generator on the GPU, we simply fit the model with the parameter `device = 'cuda'`. Here we are able to run 10 epochs in about as long as it takes to run 1 epoch on the CPU:

In [4]:
model_gpu = SygnetModel(mode = "wgan")
model_gpu.fit(data = train_data, epochs = 10, device='cuda')
synth_data = model_gpu.sample(nobs = 1000)

synth_data.head()

Epoch: 100%|██████████| 10/10 [00:08<00:00,  1.20it/s]


,x1,x2,x3,y
0,1.599184,1.105982,0.0,4.085353
1,1.200614,0.682166,0.0,4.840001
2,1.547181,0.740852,0.0,4.503998
3,1.180196,1.247587,0.0,3.763865
4,1.494661,0.882131,0.0,4.203248


## Custom architectures

The above models are trained on the default parameters and for a short number of epochs. They are therefore not well-trained (as can be seen in the resulting synthetic data).

To improve our models, we can adjust the hyperparameter settings of our model. 



In [11]:
model_custom = SygnetModel(mode = "wgan", dropout_p=0.1)

model_custom.fit(
    train_data, 
    device = 'cuda', 
    epochs = 50, 
    batch_size=512,
    learning_rate=0.001
)

synth_data = model_custom.sample(1000)

Epoch:  16%|█▌        | 16/100 [00:49<04:20,  3.10s/it]


KeyboardInterrupt: 

To assess the quality of our model, we can see how well the synthetic data outcome variable `y` is modelled by the synthetic independent variables. To do so, we can regress `y` on `x1` and `x2`. Recall that in our DGP $$y \sim \mathcal{N}(\mu = 3\times X_1 + 2 \times X_2 + 1, \sigma).$$ Therefore, we should expect the coefficient vector to be close to `[3,2,1]`. 

In [12]:
from sklearn.linear_model import LinearRegression

synth_data = model_custom.sample(100000)
reg = LinearRegression().fit(synth_data.drop(['y','x3'], axis=1), synth_data['y'])

print(f"Synthetic coefficients = {reg.coef_.tolist() + [reg.intercept_]}")


Synthetic coefficients = [3.5488946437835693, 1.5444843769073486, -0.28509402]


We find that, while the relationship is not perfectly captured, we nevertheless get data that resembles that relationship.